[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/cybersecurity/01_Real_Time_Anomaly_Detection.ipynb)

# Real-Time Anomaly Detection - Stream Processing & Temporal KGs

## Overview

This notebook demonstrates **real-time anomaly detection** using Semantica with focus on **stream ingestion**, **temporal knowledge graphs**, and **pattern detection**. The pipeline streams security logs in real-time, builds temporal knowledge graphs, and detects anomalies using pattern detection.

### Key Features

- **Stream Processing**: Emphasizes real-time log streaming and processing
- **Temporal Knowledge Graphs**: Builds temporal KGs to track events over time
- **Pattern Detection**: Uses graph patterns to identify anomalies
- **Automated Alerting**: Generates alerts for detected anomalies
- **Comprehensive Data Sources**: Multiple security RSS feeds, APIs, and databases
- **Modular Architecture**: Direct use of Semantica modules without core orchestrator

### Learning Objectives

- Ingest security data from multiple sources (RSS feeds, APIs, streams)
- Extract security entities (Logs, Events, IPs, Users, Alerts, Attacks)
- Build temporal security knowledge graphs
- Perform temporal queries and pattern detection
- Detect anomalies using graph reasoning
- Store and query security data using vector stores and graph stores

### Pipeline Flow

```mermaid
graph TD
    A[Data Ingestion] --> B[Document Parsing]
    B --> C[Text Processing]
    C --> D[Entity Extraction]
    D --> E[Relationship Extraction]
    E --> F[Deduplication]
    F --> G[Conflict Detection]
    G --> H[Temporal Knowledge Graph]
    H --> I[Embeddings]
    I --> J[Vector Store]
    H --> K[Temporal Queries]
    K --> L[Pattern Detection]
    L --> M[Reasoning & Anomaly]
    J --> N[GraphRAG Queries]
    M --> N
    H --> O[Graph Store]
    N --> P[Visualization]
    O --> P
    P --> Q[Export]
```


## Installation


In [2]:
%pip install -qU semantica networkx matplotlib plotly pandas faiss-cpu beautifulsoup4 groq sentence-transformers scikit-learn


Note: you may need to restart the kernel to use updated packages.


## Configuration & Setup


In [3]:
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "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")

# Configuration constants
EMBEDDING_DIMENSION = 384
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
TEMPORAL_GRANULARITY = "minute"


## Ingesting Security Data from Multiple Sources


In [4]:
from semantica.ingest import FeedIngestor, StreamIngestor, FileIngestor
import os
from contextlib import redirect_stderr
from io import StringIO

os.makedirs("data", exist_ok=True)

feed_sources = [
    # Security RSS Feeds
    ("US-CERT Alerts", "https://www.us-cert.gov/ncas/alerts.xml"),
    ("SANS ISC", "https://isc.sans.edu/rssfeed.xml"),
    ("Krebs on Security", "https://krebsonsecurity.com/feed/"),
    ("ThreatPost", "https://threatpost.com/feed/"),
    ("BleepingComputer", "https://www.bleepingcomputer.com/feed/"),
]

feed_ingestor = FeedIngestor()
all_documents = []

print(f"Ingesting from {len(feed_sources)} feed sources...")
for i, (feed_name, feed_url) in enumerate(feed_sources, 1):
    try:
        with redirect_stderr(StringIO()):
            feed_data = feed_ingestor.ingest_feed(feed_url, validate=False)
        
        feed_count = 0
        for item in feed_data.items:
            if not item.content:
                item.content = item.description or item.title or ""
            if item.content:
                if not hasattr(item, 'metadata'):
                    item.metadata = {}
                item.metadata['source'] = feed_name
                all_documents.append(item)
                feed_count += 1
        
        if feed_count > 0:
            print(f"  [{i}/{len(feed_sources)}] {feed_name}: {feed_count} documents")
    except Exception:
        continue

# Simulate stream ingestion (in production, use actual Kafka/WebSocket)
if not all_documents:
    security_logs = """
    2024-01-01 10:00:00 - Login attempt from IP 192.168.1.100 user admin
    2024-01-01 10:01:00 - Failed login from IP 192.168.1.100 user admin
    2024-01-01 10:02:00 - Multiple failed logins from IP 192.168.1.100 user admin
    2024-01-01 10:03:00 - Unusual activity detected from IP 192.168.1.100
    2024-01-01 10:04:00 - Alert: Potential brute force attack from IP 192.168.1.100
    2024-01-01 10:05:00 - Login attempt from IP 192.168.1.101 user test
    2024-01-01 10:06:00 - Suspicious file access from IP 192.168.1.102
    2024-01-01 10:07:00 - Multiple connection attempts from IP 192.168.1.103
    """
    with open("data/security_logs.txt", "w") as f:
        f.write(security_logs)
    file_ingestor = FileIngestor()
    all_documents = file_ingestor.ingest("data/security_logs.txt")

documents = all_documents
print(f"Ingested {len(documents)} documents")


Ingesting from 5 feed sources...


Status,Action,Module,Submodule,File,Time
✅,Semantica is extracting,🎯 semantic_extract,NERExtractor,-,0.55s
✅,Semantica is extracting,🎯 semantic_extract,RelationExtractor,-,0.05s
✅,Semantica is resolving,⚠️ conflicts,ConflictDetector,-,0.00s
✅,Semantica is building,🧠 kg,GraphBuilder,-,350.71s
🔄,Semantica is building,🧠 kg,EntityResolver,-,84.09s
✅,Semantica is deduplicating,🔄 deduplication,DuplicateDetector,-,0.08s
✅,Semantica is deduplicating,🔄 deduplication,SimilarityCalculator,-,0.02s
✅,Semantica is deduplicating,🔄 deduplication,EntityMerger,-,0.13s
✅,Semantica is deduplicating,🔄 deduplication,MergeStrategyManager,-,0.03s
✅,Semantica is indexing,📊 vector_store,VectorStore,-,0.01s


  [1/5] US-CERT Alerts: 10 documents
  [2/5] SANS ISC: 10 documents
  [3/5] Krebs on Security: 10 documents
  [4/5] ThreatPost: 10 documents
Ingested 40 documents


In [5]:
from semantica.parse import DocumentParser

parser = DocumentParser()

print(f"Parsing {len(documents)} documents...")
parsed_documents = []
for i, doc in enumerate(documents, 1):
    try:
        parsed = parser.parse(
            doc.content if hasattr(doc, 'content') else str(doc),
            content_type="text"
        )
        parsed_documents.append(parsed)
    except Exception:
        parsed_documents.append(doc)
    if i % 50 == 0 or i == len(documents):
        print(f"  Parsed {i}/{len(documents)} documents...")

documents = parsed_documents


Parsing 40 documents...
  Parsed 40/40 documents...


## Normalizing and Chunking Security Logs


In [6]:
from semantica.normalize import TextNormalizer
from semantica.split import TextSplitter

normalizer = TextNormalizer()
# Use sentence chunking for log line boundaries (structured logs)
splitter = TextSplitter(method="sentence", chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

print(f"Normalizing {len(documents)} documents...")
normalized_documents = []
for i, doc in enumerate(documents, 1):
    normalized_text = normalizer.normalize(
        doc.content if hasattr(doc, 'content') else str(doc),
        clean_html=True,
        normalize_entities=True,
        remove_extra_whitespace=True,
        lowercase=False
    )
    normalized_documents.append(normalized_text)
    if i % 50 == 0 or i == len(documents):
        print(f"  Normalized {i}/{len(documents)} documents...")

print(f"Chunking {len(normalized_documents)} documents...")
chunked_documents = []
for i, doc_text in enumerate(normalized_documents, 1):
    try:
        with redirect_stderr(StringIO()):
            chunks = splitter.split(doc_text)
        chunked_documents.extend(chunks)
    except Exception:
        simple_splitter = TextSplitter(method="recursive", chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        chunks = simple_splitter.split(doc_text)
        chunked_documents.extend(chunks)
    if i % 50 == 0 or i == len(normalized_documents):
        print(f"  Chunked {i}/{len(normalized_documents)} documents ({len(chunked_documents)} chunks so far)")

print(f"Created {len(chunked_documents)} chunks from {len(normalized_documents)} documents")


Normalizing 40 documents...
  Normalized 40/40 documents...
Chunking 40 documents...
  Chunked 40/40 documents (307 chunks so far)
Created 307 chunks from 40 documents


## Extracting Security Entities


In [11]:
from semantica.semantic_extract import NERExtractor

security_patterns = {
    "IP": r"\b(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b",
    "User": r"\buser\s+([a-zA-Z0-9_\-\.]+)\b",
    "Alert": r"\b(?:alert|warning|alarm):\s*([^\n\.]+)|\b(?:alert|warning|alarm)\s+(?:detected|triggered|generated|raised)\b",
    "Event": r"\b(?:login|access|connection|request|attempt|failed|successful|suspicious|unusual)\s+(?:event|attempt|request|activity|access)\b",
    "Log": r"\b\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}\s+-\s+([^\n]+)",
    "Attack": r"\b(?:attack|breach|intrusion|exploit|malware|virus|ransomware|phishing|brute\s+force|ddos)\b",
}

entity_extractor = NERExtractor(method="regex", patterns=security_patterns)

all_entities = []
print(f"Extracting entities from {len(chunked_documents)} chunks...")
for i, chunk in enumerate(chunked_documents, 1):
    chunk_text = chunk.text if hasattr(chunk, 'text') else str(chunk)
    try:
        entities = entity_extractor.extract_entities(
            chunk_text,
            entity_types=["Log", "Event", "IP", "User", "Alert", "Attack"]
        )
        all_entities.extend(entities)
    except Exception:
        continue
    
    if i % 20 == 0 or i == len(chunked_documents):
        print(f"  Processed {i}/{len(chunked_documents)} chunks ({len(all_entities)} entities found)")

ips = [e for e in all_entities if e.label == "IP" or "ip" in e.label.lower()]
users = [e for e in all_entities if e.label == "User" or "user" in e.label.lower()]
alerts = [e for e in all_entities if e.label == "Alert" or "alert" in e.label.lower()]

print(f"Extracted {len(ips)} IPs, {len(users)} users, {len(alerts)} alerts")


Extracting entities from 307 chunks...
  Processed 20/307 chunks (6 entities found)
  Processed 40/307 chunks (10 entities found)
  Processed 60/307 chunks (15 entities found)
  Processed 80/307 chunks (26 entities found)
  Processed 100/307 chunks (61 entities found)
  Processed 120/307 chunks (86 entities found)
  Processed 140/307 chunks (98 entities found)
  Processed 160/307 chunks (116 entities found)
  Processed 180/307 chunks (144 entities found)
  Processed 200/307 chunks (160 entities found)
  Processed 220/307 chunks (198 entities found)
  Processed 240/307 chunks (232 entities found)
  Processed 260/307 chunks (247 entities found)
  Processed 280/307 chunks (257 entities found)
  Processed 300/307 chunks (269 entities found)
  Processed 307/307 chunks (273 entities found)
Extracted 3 IPs, 28 users, 0 alerts


## Extracting Security Relationships


In [16]:
from semantica.semantic_extract import RelationExtractor

# Filter entities to only meaningful security entities
filtered_entities = [
    e for e in all_entities 
    if e.label in ["IP", "User", "Alert", "Attack", "Event", "Log"] 
    and len(e.text) > 2
    and e.text.lower() not in ["to", "from", "should", "would", "choices", "connects"]
]

relation_extractor = RelationExtractor(
    method="cooccurrence",
    max_distance=60,
    confidence_threshold=0.6
)

# Deduplicate relationships
seen_relationships = set()
all_relationships = []
print(f"Extracting relationships from {len(chunked_documents)} chunks using {len(filtered_entities)} filtered entities...")
for i, chunk in enumerate(chunked_documents, 1):
    chunk_text = chunk.text if hasattr(chunk, 'text') else str(chunk)
    try:
        relationships = relation_extractor.extract_relations(
            chunk_text,
            entities=filtered_entities,
            relation_types=["from", "attempts", "triggers", "detects", "associated_with", "causes"]
        )
        # Deduplicate based on subject, predicate, object
        for rel in relationships:
            rel_key = (rel.subject.text, rel.predicate, rel.object.text)
            if rel_key not in seen_relationships:
                seen_relationships.add(rel_key)
                all_relationships.append(rel)
    except Exception:
        continue
    
    if i % 20 == 0 or i == len(chunked_documents):
        print(f"  Processed {i}/{len(chunked_documents)} chunks ({len(all_relationships)} relationships found)")

print(f"Extracted {len(all_relationships)} relationships")


Extracting relationships from 307 chunks using 267 filtered entities...
  Processed 20/307 chunks (534 relationships found)
  Processed 40/307 chunks (534 relationships found)
  Processed 60/307 chunks (534 relationships found)
  Processed 80/307 chunks (534 relationships found)
  Processed 100/307 chunks (534 relationships found)
  Processed 120/307 chunks (534 relationships found)
  Processed 140/307 chunks (534 relationships found)
  Processed 160/307 chunks (534 relationships found)
  Processed 180/307 chunks (534 relationships found)
  Processed 200/307 chunks (534 relationships found)
  Processed 220/307 chunks (534 relationships found)
  Processed 240/307 chunks (534 relationships found)
  Processed 260/307 chunks (534 relationships found)
  Processed 280/307 chunks (534 relationships found)
  Processed 300/307 chunks (534 relationships found)
  Processed 307/307 chunks (534 relationships found)
Extracted 534 relationships


## Detecting Security Conflicts

- **Using entity-wide conflict detection** to identify all types of conflicts (value, type, relationship, temporal) across security entities from multiple sources. 
- **Voting resolution strategy** selects the majority consensus value, ensuring reliability through multi-source agreement for security event data.


In [21]:
from semantica.conflicts import ConflictDetector, ConflictResolver

conflict_detector = ConflictDetector()
conflict_resolver = ConflictResolver()

# Convert entities to dictionaries for conflict detection
entity_dicts = [
    {
        "id": e.text,
        "text": e.text,
        "label": e.label,
        "type": e.label,
        "confidence": e.confidence if hasattr(e, 'confidence') else 1.0,
        "metadata": e.metadata if hasattr(e, 'metadata') else {}
    }
    for e in all_entities
]

# Convert relationships to dictionaries for conflict detection
relationship_dicts = [
    {
        "id": f"{r.subject.text}_{r.predicate}_{r.object.text}",
        "source_id": r.subject.text,
        "target_id": r.object.text,
        "type": r.predicate,
        "subject": r.subject.text,
        "object": r.object.text,
        "predicate": r.predicate,
        "confidence": r.confidence if hasattr(r, 'confidence') else 1.0
    }
    for r in all_relationships
]

print(f"Detecting conflicts in {len(entity_dicts)} entities and {len(relationship_dicts)} relationships...")

# Detect entity conflicts (value, type, temporal)
value_conflicts = conflict_detector.detect_value_conflicts(entity_dicts, property_name="label")
type_conflicts = conflict_detector.detect_type_conflicts(entity_dicts)
temporal_conflicts = conflict_detector.detect_temporal_conflicts(entity_dicts)
entity_conflicts = value_conflicts + type_conflicts + temporal_conflicts

# Detect relationship conflicts
relationship_conflicts = conflict_detector.detect_relationship_conflicts(relationship_dicts)

# Combine all conflicts
all_conflicts = entity_conflicts + relationship_conflicts

print(f"Detected {len(all_conflicts)} conflicts ({len(entity_conflicts)} entity, {len(relationship_conflicts)} relationship)")

if all_conflicts:
    print(f"Resolving conflicts using voting strategy...")
    resolved = conflict_resolver.resolve_conflicts(
        all_conflicts,
        strategy="voting"
    )
    print(f"Resolved {len(resolved)} conflicts")
else:
    print("No conflicts detected")


Detecting conflicts in 273 entities and 534 relationships...


Detected 0 conflicts (0 entity, 0 relationship)
No conflicts detected


## Building Temporal Security Knowledge Graph


In [23]:
from semantica.kg import GraphBuilder

graph_builder = GraphBuilder(
    merge_entities=True,
    resolve_conflicts=True,
    entity_resolution_strategy="fuzzy",
    enable_temporal=True,
    temporal_granularity=TEMPORAL_GRANULARITY
)

print(f"Building knowledge graph...")
kg_sources = [{
    "entities": [{"text": e.text, "type": e.label, "confidence": e.confidence if hasattr(e, 'confidence') else 1.0} for e in all_entities],
    "relationships": [{"source": r.subject.text, "target": r.object.text, "type": r.predicate, "confidence": r.confidence if hasattr(r, 'confidence') else 1.0} for r in all_relationships]
}]

kg = graph_builder.build(kg_sources)

entities_count = len(kg.get('entities', []))
relationships_count = len(kg.get('relationships', []))
print(f"Graph: {entities_count} entities, {relationships_count} relationships")


Building knowledge graph...
Processing 273 entities, 534 relationships (807 total)...
  Entities: 100/273 (36.6%) | ETA: 3.5m | Rate: 0.8/s
  Entities: 200/273 (73.3%) | ETA: 1.5m | Rate: 0.8/s
  Entities: 273/273 (100.0%) | ETA: 0.0s | Rate: 0.8/s
  Relationships: 100/534
  Relationships: 200/534
  Relationships: 300/534
  Relationships: 400/534
  Relationships: 500/534
  Relationships: 534/534
Resolving 30 entities...
✅ Resolved to 3 unique entities (8.32s)
Building graph structure...
✅ Graph structure built (0.00s)

✅ Knowledge Graph Build Complete
   Entities: 3
   Relationships: 534
   Total time: 350.71s
Graph: 3 entities, 534 relationships


## Generating Embeddings for Events and IPs


In [24]:
from semantica.embeddings import EmbeddingGenerator

embedding_gen = EmbeddingGenerator(
    provider="sentence_transformers",
    model=EMBEDDING_MODEL
)

event_entities = [e for e in all_entities if e.label in ["Event", "Log", "Alert"]]
print(f"Generating embeddings for {len(event_entities)} events and {len(ips)} IPs...")
event_texts = [e.text for e in event_entities]
event_embeddings = embedding_gen.generate_embeddings(event_texts)

ip_texts = [ip.text for ip in ips]
ip_embeddings = embedding_gen.generate_embeddings(ip_texts)

print(f"Generated {len(event_embeddings)} event embeddings and {len(ip_embeddings)} IP embeddings")


fastembed not available. Install with: pip install fastembed. Using fallback embedding method.


Generating embeddings for 6 events and 3 IPs...
Generated 6 event embeddings and 3 IP embeddings


## Populating Vector Store


In [25]:
from semantica.vector_store import VectorStore

vector_store = VectorStore(backend="faiss", dimension=EMBEDDING_DIMENSION)

print(f"Storing {len(event_embeddings)} event vectors and {len(ip_embeddings)} IP vectors...")
event_ids = vector_store.store_vectors(
    vectors=event_embeddings,
    metadata=[{"type": "event", "name": e.text, "label": e.label} for e in event_entities]
)

ip_ids = vector_store.store_vectors(
    vectors=ip_embeddings,
    metadata=[{"type": "ip", "name": ip.text, "label": ip.label} for ip in ips]
)

print(f"Stored {len(event_ids)} event vectors and {len(ip_ids)} IP vectors")


fastembed not available. Install with: pip install fastembed. Using fallback embedding method.


Storing 6 event vectors and 3 IP vectors...
Stored 6 event vectors and 3 IP vectors


## Temporal Graph Queries


In [26]:
from semantica.kg import TemporalGraphQuery

temporal_query = TemporalGraphQuery(
    enable_temporal_reasoning=True,
    temporal_granularity=TEMPORAL_GRANULARITY
)

query_results = temporal_query.query_at_time(
    kg,
    query={"type": "Alert"},
    at_time="2024-01-01 10:04:00"
)

evolution = temporal_query.analyze_evolution(kg)
temporal_patterns = temporal_query.detect_temporal_patterns(kg, pattern_type="sequence")

print(f"Temporal queries: {len(query_results)} alerts at query time")
print(f"Temporal patterns detected: {len(temporal_patterns)}")


AttributeError: 'TemporalGraphQuery' object has no attribute 'detect_temporal_patterns'

## Detecting Anomaly Patterns


In [ ]:
from semantica.kg import GraphAnalyzer

graph_analyzer = GraphAnalyzer()

# Detect suspicious IPs
suspicious_ips = []
for entity in kg.get("entities", []):
    if entity.get("type") == "IP":
        related_rels = [r for r in kg.get("relationships", []) 
                        if r.get("source") == entity.get("id") or r.get("target") == entity.get("id")]
        if any("alert" in str(r.get("type", "")).lower() or "attack" in str(r.get("type", "")).lower() 
               for r in related_rels):
            suspicious_ips.append(entity)

# Detect anomaly patterns (multiple failed logins, unusual activity)
anomaly_patterns = []
for ip in ips[:10]:
    ip_name = ip.text
    paths = graph_analyzer.find_paths(
        kg,
        source=ip_name,
        target_type="Alert",
        max_hops=2
    )
    if len(paths) > 0:
        anomaly_patterns.append({
            'ip': ip_name,
            'alert_count': len(paths),
            'pattern': 'suspicious_activity'
        })

print(f"Pattern detection: {len(anomaly_patterns)} anomaly patterns found")
print(f"Suspicious IPs: {len(suspicious_ips)}")


## Reasoning and Anomaly Detection


In [ ]:
from semantica.reasoning import Reasoner

reasoner = Reasoner()

reasoner.add_rule("IF IP attempts Event AND Event type failed_login AND Event count > 3 THEN IP triggers Alert")
reasoner.add_rule("IF User from IP AND IP triggers Alert THEN User associated_with Alert")

inferred_facts = reasoner.infer_facts(kg)

anomaly_paths = reasoner.find_paths(
    kg,
    source_type="IP",
    target_type="Alert",
    max_hops=2
)

print(f"Inferred {len(inferred_facts)} facts")
print(f"Found {len(anomaly_paths)} anomaly paths")


## Storing Security Knowledge Graph (Optional)


In [ ]:
from semantica.graph_store import GraphStore

# Optional: Store to persistent graph database
# graph_store = GraphStore(backend="neo4j", uri="bolt://localhost:7687", user="neo4j", password="password")
# graph_store.store_graph(kg)

print("Graph store configured (commented out for demo)")


## GraphRAG: Hybrid Vector + Graph Queries


In [ ]:
from semantica.context import AgentContext
from semantica.llms import Groq
import os

context = AgentContext(vector_store=vector_store, knowledge_graph=kg)

# Initialize LLM provider
llm_provider = Groq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

query = "What IPs are associated with security alerts?"
result = context.query_with_reasoning(
    query=query,
    llm_provider=llm_provider,
    max_results=10,
    max_hops=2
)

print(f"GraphRAG Query with Reasoning: '{query}'\n")
print("=" * 80)
print(f"\nGenerated Response:\n{result['response']}\n")
print("=" * 80)
if result.get('reasoning_path'):
    print(f"\nReasoning Path:\n{result['reasoning_path']}\n")
print(f"Confidence: {result.get('confidence', 0):.3f}")
print(f"Sources Used: {result.get('num_sources', 0)}")
print(f"Reasoning Paths Found: {result.get('num_reasoning_paths', 0)}")


## Visualizing the Temporal Security Knowledge Graph


In [ ]:
from semantica.visualization import KGVisualizer

visualizer = KGVisualizer()
visualizer.visualize(
    kg,
    output_path="anomaly_detection_kg.html",
    layout="temporal",
    node_size=20
)

print("Visualization saved to anomaly_detection_kg.html")


## Exporting Results


In [ ]:
from semantica.export import GraphExporter

exporter = GraphExporter()
exporter.export(kg, output_path="anomaly_detection_kg.json", format="json")
exporter.export(kg, output_path="anomaly_detection_kg.graphml", format="graphml")
exporter.export(kg, output_path="anomaly_detection_alerts.csv", format="csv")

print("Exported knowledge graph to JSON, GraphML, and CSV formats")
